In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
wifi_all_pd = pd.read_parquet('../data/raw/wifi_data/')

In [ ]:
wifi_all_pd = gpd.GeoDataFrame(wifi_all_pd, geometry=gpd.points_from_xy(wifi_all_pd["longitude"], wifi_all_pd["latitude"]))
wifi_all_pd.dtypes

In [ ]:
sf = gpd.read_file("../data/taxi_zones/taxi_zones.shp")
zones = pd.read_csv("../data/taxi_zones/taxi+_zone_lookup.csv")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gdf = gpd.GeoDataFrame(
    pd.merge(zones, sf, on='LocationID', how='inner')
)
wifi_all_gpd = gpd.sjoin(wifi_all_pd, gdf)

In [ ]:
wifi_all_pd.head()

In [ ]:
import numpy as np

proportions = wifi_all_pd[['LocationID', 'wifi_id','borough']] \
    .groupby(['LocationID', 'borough']) \
    .agg(
    {
        'wifi_id': 'count',  # sum over total amount earned
    }
) \
    .rename({'wifi_id': 'num_wifi'}, axis=1)\
    .rename({'LocationID': 'PULocationID'}, axis=1)

proportions["log_num_wifi"] = np.log(proportions["num_wifi"])
proportions.head()